# SWEN90006 Tutorial 7 Solution

**NOTE** You are expected to prepare for this tutorial by sketching
answers to the tasks and questions before attending the tutorial.

## Introduction
The aim of this tutorial is to get you thinking about software security
and vulnerabilities, and the applicability of different kinds of
security testing.

As a first step, think about what security testing is, and why we would
want to perform security testing on our software.

**Solution:**

1. Ordinary program testing seeks to uncover program faults, exhibited by failures: when the program's observable behaviour differs from what it was intended or required to do. In contrast, security testing seeks to uncover security *vulnerabilities*: typically extra functionality that allows an attacker to do something unwanted.

    This "something unwanted" could include the attacker:

    -   Causing the program to crash (perhaps denying service,
        i.e. *availability*, to other users);

    -   Causing the program to corrupt data (violating data *integrity*);

    -   Causing the program to reveal sensitive data (violating data
        *confidentiality*);

    -   Causing the program to take an excessive amount of time to execute
        (degrading performance and possibly affecting availability for other
        users);

    -   Causing the program to execute attacker-controlled functionality (in
        the worst case, arbitrary attacker-supplied code).


2. The prevalence of security *patching*, where software upgrades are
periodically released to address security vulnerabilities, indicates how
difficult it is to program software without security vulnerabilities
and, therefore, the importance of testing for them. The potential cost
of an attacker exploiting these vulnerabilities, in an age where
software routinely manages and processes huge bodies of users' personal
information, only makes the case for security testing stronger.


3. While the unwanted functionality is usually extra functionality,
unintended by the program's specification or requirements, remember that
by testing for common kinds of unwanted behaviour (e.g. program crashes,
undefined behaviour, information leaks etc.), security testing (as with
ordinary testing) can also uncover vulnerabilities that exist in a
program's specification or design.

## The Bitmap File Format

BMP is an historical image file format that we will use in this
tutorial. We will consider a simple class of BMP files whose format is
as follows. (Specifically, we consider here BMP files with no
compression, and in which each pixel is 32-bits wide in order to avoid
issues of padding; see
<http://www.fastgraph.com/help/bmp_header_format.html> and
<https://en.wikipedia.org/wiki/BMP_file_format> for more details.)

| Offset | Size (in bytes)              | Description                                                        |
|--------|------------------------------|--------------------------------------------------------------------|
| 0      | 1                            | first byte of signature, must be 0x42 (the ASCII character 'B')    |
| 1      | 1                            | second byte of signature, must be 0x4D (the ASCII character 'M')   |
| 2      | 4                            | size of the BMP file in bytes (unreliable, ignored)                |
| 6      | 2                            | Must be zero                                                       |
| 8      | 2                            | Must be zero                                                       |
| 10     | 4                            | Must be the value 54 (i.e. 0x00000036)                            |
| 14     | 4                            | Must be the value 40 (i.e. 0x00000028)                            |
| 18     | 4                            | *Width* (image width in pixels, as signed integer)            |
| 22     | 4                            | *Height* (image height in pixels, as signed integer)          |
| 26     | 2                            | Must be one                                                        |
| 28     | 2                            | Number of bits per pixel (must be 32)                              |
| 30     | 4                            | Compression type (must be 0 = no compression)                      |
| 34     | 4                            | Size of image data in bytes (must be 4\**Width*\**Height*) |
| 38     | 4                            | unreliable (ignored)                                               |
| 42     | 4                            | unreliable (ignored)                                               |
| 46     | 4                            | Must be zero                                                       |
| 50     | 4                            | Must be zero                                                       |
| 54     | 4\**Width*\**Height* | Pixel data, laid out in rows                                       |

The first byte (offset 0) of a valid BMP file is the character 'B'; the
second byte (offset 1) is the character 'M'. The 3rd to 6th bytes
(offsets 2 to 5 inclusive) indicate the total length of the BMP file but
are unreliable in practice and so let us assume that they are ignored by
all BMP parsing code. The 7th and 8th bytes (offsets 6 and 7) are
interpreted as a 2-byte integer that must be zero, i.e. each of these
bytes must be zero. The same is true for the 9th and 10th bytes (offsets
8 and 9), and so on.

## Your Tasks


### Question 1
Imagine you are choosing a value for each of the fields in the table
above *in order*, i.e. you first choose a value for the first byte of
the file, then choose a value for the second byte of the file, then for
following 4-bytes, and so on. For each field, identify the total number
of valid values there are to choose from, assuming you have already
chosen values for all fields that have come before.

**Solution**:

| Offset | Size (in bytes)              | Description                                                    | Choices                                                                              |
|--------|------------------------------|----------------------------------------------------------------|--------------------------------------------------------------------------------------|
| 0      | 1                            | must be 0x42                                                   | 1                                                                                    |
| 1      | 1                            | must be 0x4D                                                   | 1                                                                                    |
| 2      | 4                            | ignored, so we can choose anything                             | $2^{32}$                                                                             |
| 6      | 2                            | Must be zero                                                   | 1                                                                                    |
| 8      | 2                            | Must be zero                                                   | 1                                                                                    |
| 10     | 4                            | Must be the value 54                                           | 1                                                                                    |
| 14     | 4                            | Must be the value 40                                           | 1                                                                                    |
| 18     | 4                            | Width (signed int), must be in $1 \ldots 2^{31} - 1$  | $2^{31} - 1$                                                                         |
| 22     | 4                            | Height (signed int), must be in $1 \ldots 2^{31} - 1$ | $2^{31} - 1$                                                                         |
| 26     | 2                            | Must be one                                                    | 1                                                                                    |
| 28     | 2                            | Must be 32                                                     | 1                                                                                    |
| 30     | 4                            | Must be 0                                                      | 1                                                                                    |
| 34     | 4                            | Must be 4\*Width\*Height                           | 1                                                                                    |
| 38     | 4                            | ignored, so can be anything                                    | $2^{32}$                                                                             |
| 42     | 4                            | ignored, so can be anything                                    | $2^{32}$                                                                             |
| 46     | 4                            | Must be zero                                                   | 1                                                                                    |
| 50     | 4                            | Must be zero                                                   | 1                                                                                    |
| 54     | 4\**Width*\**Height* | Pixel data, can be anything                                    | $$~~~256^{4*\mathit{Width}*\mathit{Height}}~~~=~~~(2^{32})^{\mathit{Width}*\mathit{Height}}~~$$ |


### Question 2
The BMP header (i.e. everything excluding the pixel data) as described
above has a fixed length of 54 bytes. Using the answer from the previous
question or otherwise, what is the probability that a (uniformly)
randomly generated string of 54 bytes is a valid BMP header?

**Solution:**

$$\begin{array}{l}
= \frac{1}{2^8} \cdot \frac{1}{2^8} \cdot 1 \cdot \frac{1}{2^{16}} \cdot \frac{1}{2^{16}} \cdot \frac{1}{2^{32}} \cdot \frac{1}{2^{32}} \cdot \frac{2^{31} - 1}{2^{32}} \cdot \frac{2^{31} - 1}{2^{32}} \cdot \frac{1}{2^{16}} \cdot \frac{1}{2^{16}} \cdot \frac{1}{2^{32}} \cdot \frac{1}{2^{32}} \cdot 1 \cdot 1 \cdot \frac{1}{2^{32}} \cdot \frac{1}{2^{32}} \\ \\
= \frac{2^{62} - 2^{32} - 1}{2^{336}}  \\ \\
\approx \frac{2^{62}}{2^{336}}  \\ \\
= \frac{1}{2^{274}}  \\ \\
\approx 3 \times 10^{-83}  \\
\end{array}$$

### Question 3
Suppose you have a valid 54-byte header and you mutate an arbitrary
(uniformly randomly chosen) byte in the header to a new value (different
from its original value). What is the probability of producing a valid
header?

**Solution:**
We compute the probability for each byte choice, noting that for the
4-byte signed integers *Width* and *Height*, they can't be modified
without breaking the constraint that the size of the image data in bytes
(offsets 34--37) must be equal to 4\**Width*\**Height*. Likewise those
offsets cannot be modified either.

| Byte offset | Probability | Notes        |
|--------------------|----------------------|-----------------------|
| 0                  | 0                    |                       |
| 1                  | 0                    |                       |
| 2                  | 1                    |                       |
| 3                  | 1                    |                       |
| 4                  | 1                    |                       |
| 5                  | 1                    |                       |
| 6                  | 0                    |                       |
| 7                  | 0                    |                       |
| 8                  | 0                    |                       |
| $\ldots$             | $\ldots$               |                       |
| 17                 | 0                    |                       |
| 18                 | 0                    | only one valid choice |
| 19                 | $\ldots$               |                       |
| 25                 | 0                    | only one valid choice |
| 26                 | 0                    |                       |
| 27                 | $\ldots$               |                       |
| 37                 | 0                    |                       |
| 38                 | 1                    | All choices are valid |
| 39                 | $\ldots$               | All choices are valid |
| 45                 | 1                    | All choices are valid |
| 46                 | 0                    |                       |
| 47                 | $\ldots$               |                       |
| 53                 | 0                    |                       |


Letting $P(i)$ denote the probability from the above table for
byte-offset $i$, then the total probability is:

$$\sum\limits_i \frac{P(i)}{54} = \frac{\sum\limits_i P(i)}{54}$$


$$\sum\limits_i P(i) = 12$$


Thus the total probability is:

$$\frac{12}{54} \approx 0.222$$

So, finally, there is a 22.2% chance of a mutation in one byte still
producing a valid header.

### Question 4
Imagine you had to write a fuzzer to fuzz some BMP processing code that
processed BMP files of the format described above. If you had to choose
between generating completely random inputs vs. using random mutation on
existing BMP files, which strategy would you choose?

**Solution:**

Given the above, mutation is likely to produce more valid BMP headers,
so should be preferable. Specifically, it is likely to produce inputs
that achieve greater code coverage. Inputs generated by entirely random
strings of bytes are all likely to be invalid BMP files and so we might
expect them to all take the code paths taken only by invalid files.

On the other hand, up to $\approx 22\%$ of inputs generated using
mutation we can expect to be valid (depending on the mutation strategy,
this percentage might quickly decrease, though) and therefore to take
code paths that are almost impossible to trigger using random inputs
alone. Mutation will still generate plenty of invalid BMP headers, so
will still exercise code paths taken by invalid BMP files. Therefore we
might reasonably expect that the inputs produced by random mutation will
take a superset of the paths of those produced by entirely random
inputs.
